In [118]:
# import numpy as np
# import pandas as pd
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.metrics.pairwise import linear_kernel
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.feature_extraction.text import TfidfVectorizer
# from collections import Counter
# import mysql.connector
# import warnings
# warnings.filterwarnings("ignore")



# # join and concatenate inputted columns
# def create_soup(x, features):
#     soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
#     return ' '.join(soup_parts)

# # count number of likeable elements for film
# def count_likeable(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if col is not None)
#     cast = len(row['cast'].split(','))
#     return atts+cast

# # get user loved films from db
# def get_loved_films(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     rows = mycursor.fetchall()

#     tconst_list = [tconst[0] for tconst in rows]

#     mycursor.close()
#     mydb.close()

#     loved_films_df = data[data['tconst'].isin(tconst_list)]

#     return loved_films_df

# # get user liked attributes from db
# def get_liked_attributes(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     attribute_fetch = mycursor.fetchall()

#     tconst_list = [row[1] for row in attribute_fetch]
#     attribute_bin = [row[1:] for row in attribute_fetch]

#     mycursor.close()
#     mydb.close()

#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     liked_attributes_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst, att_bin in zip(tconst_list, attribute_bin):
#         row_values = [tconst]  # Initialize row values with tconst
#         for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
#             film_att = data[data['tconst'] == tconst]

#             if liked == 1:
#                 row_values.append(film_att[attribute].values[0])
#             else:
#                 row_values.append(None) 
#         liked_attributes_df.loc[x] = row_values
#         x+=1

#     return liked_attributes_df

# # get user liked cast from db
# def get_liked_cast(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     cast_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     cast = [row[1:] for row in cast_fetch]
#     liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

#     return liked_cast_df

# # get user watchlist from db
# def get_watchlist(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))


#     watchlist_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     tconst_list = [row[1] for row in watchlist_fetch]
#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     watchlist_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst in tconst_list:
#         watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
#         x+=1

#     return watchlist_df

# # calculate % of liked attributes of film in profile
# def calculate_percentage(row):
#     total_attributes = row['total_likeable']
#     total_liked = row['num_liked_atts']
#     percentage = total_liked / total_attributes
#     return percentage 

# # count liked attributes for film
# def count_liked(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
#     cast = 0
#     if not pd.isna(row['cast']):
#         cast = len(row['cast'].split(','))
    
#     return atts + cast

# # CREATE USER PROFILE
# def get_user_profile(user_id):
#     # get user film preferences from db
#     lovedFilms = get_loved_films(user_id)
#     likedAtt = get_liked_attributes(user_id)
#     likedCast = get_liked_cast(user_id)
#     watchlist = get_watchlist(user_id)

#     # merge liked cast with liked attribute table
#     likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
#     likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
#     likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

#     # format user_profile dataframe
#     merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
#     user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
#     user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
#     user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

#     # initialise columns to calculate likeage
#     user_profile['num_liked_atts'] = 0
#     user_profile['likeage'] = 0.0
#     # count total liked attributes for each film in profile
#     for index, film in user_profile.iterrows():
#         tconst = film['tconst']
#         user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

#     # calculate likeage for each film in profile
#     user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
#     user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

#     return (user_profile, lovedFilms)

# # split user profile into likes by group
# def collate_liked_groups(user_profile):
#     group_dataframes = []

#     # Define dictionary to map group names to corresponding columns
#     group_columns = {
#         'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
#         'liked_cast': ['cast', 'likeage'],
#         'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
#         'liked_genre': ['genres', 'likeage'],
#         'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
#     }

#     # Iterate over the dictionary and create group dataframes
#     for group_name, columns in group_columns.items():
#         group_df = user_profile[columns].copy()
#         # Drop rows with all NaN values in specified columns
#         group_df = group_df.dropna(subset=columns[:-1], how='all')
#         group_dataframes.append(group_df)

#     return group_dataframes

# # cosine similarity vector with tf-idf between films in row and column
# def create_similarity_vector(row, column):
#     tfidf = TfidfVectorizer(stop_words='english')
#     row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
#     row_soup = row_soup_temp.fillna('')
#     row_matrix = tfidf.fit_transform(row_soup)

#     column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
#     column_soup = column_soup_temp.fillna('')
#     column_matrix = tfidf.transform(column_soup)

#     return linear_kernel(row_matrix, column_matrix)

# # euclidean distance vector for numerical attributes of film
# def create_euclidean_vector(row, column):
#     scaler = MinMaxScaler()
#     row = row.fillna('')
#     column = column.fillna(0)

#     row_normalized = scaler.fit_transform(row)
#     column_normalized = scaler.transform(column)
#     distances = euclidean_distances(row_normalized, column_normalized)
#     euclidean_matrix = 1 / (1 + distances)
#     return euclidean_matrix

# # get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
# def get_similar_films(vector, N):
#     filtered_vector = vector[~data['tconst'].isin(lovedFilms['tconst'])] #don't recommend film the user has already loved
#     mean_similarity = np.mean(filtered_vector, axis=1)
#     sorted_indices = np.argsort(mean_similarity)[::-1]
#     top_N = sorted_indices[:N]
#     return data.iloc[top_N]

# # function to calculate unified recommendations
# def get_unified_recommendations(user_profile_groups, similarity_vectors, N):

#     weighted_scores = {}
    
#     # scale similarity score in respective vector based on likeage of feature
#     for group, attributes in user_profile_groups.items():
#         similarity_vector = similarity_vectors[group][~data['tconst'].isin(user_profile['tconst'])] #don't recommend films already in profile
#         likeage_array = np.array(list(attributes['likeage'].tolist()))
#         weighted_similarity = similarity_vector * likeage_array

#         # meta columns have higher similarity since its numerical so adjust for this
#         if group == 'meta':
#             scaling_factor = 1 - weighted_similarity
#             weighted_similarity *= scaling_factor

#         # dictionary of groups and weighted similarity vectors 
#         weighted_scores[group] = weighted_similarity

#      # combine weighted similarity scores across all groups
#     combined_scores = np.sum(list(weighted_scores.values()), axis=0)

#     # calculate mean similarity scores
#     mean_similarity = np.mean(combined_scores, axis=1)

#     # sort the mean similarity scores and retrieve the top N indices
#     sorted_indices = np.argsort(mean_similarity)[::-1][:N]

#     return data.iloc[sorted_indices]


# data = pd.read_json('webpage/films.json')
# data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)
# data.head()


# # get update user profile and loved films
# get_profile = get_user_profile('57')
# user_profile = get_profile[0]
# lovedFilms = get_profile[1]

# # get groups of liked attributes
# grouped_likes = collate_liked_groups(user_profile)
# liked_plot = grouped_likes[0]
# liked_cast = grouped_likes[1]
# liked_crew = grouped_likes[2]
# liked_genre = grouped_likes[3]
# liked_meta = grouped_likes[4]

# # Define the list of features excluding 'likeage'
# plot_features = [col for col in liked_plot.columns if col != 'likeage']
# crew_features = [col for col in liked_crew.columns if col != 'likeage']
# cast_features = [col for col in liked_cast.columns if col != 'likeage']
# genre_features = [col for col in liked_genre.columns if col != 'likeage']
# meta_features = [col for col in liked_meta.columns if col != 'likeage']

# # Call the function with the selected columns
# plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
# crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
# cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
# genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
# meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])


# # Similarity vectors for each group
# similarity_vectors = {
#     'plot': plot_matrix,
#     'cast': cast_matrix,
#     'crew': crew_matrix,
#     'genre': genre_matrix,
#     'meta': meta_matrix
# }

# user_profile_groups = {
#     'plot': liked_plot,
#     'cast': liked_cast,
#     'crew': liked_crew,
#     'genre': liked_genre,
#     'meta': liked_meta
# }


# get_unified_recommendations(user_profile_groups, similarity_vectors, 20)



In [119]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import mysql.connector
import warnings
warnings.filterwarnings("ignore")

data = pd.read_json('webpage/films.json')

# join and concatenate inputted columns
def create_soup(x, features):
    soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
    return ' '.join(soup_parts)

# count number of likeable elements for film
def count_likeable(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if col is not None)
    cast = len(row['cast'].split(','))
    return atts+cast

# get user loved films from db
def get_loved_films(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    rows = mycursor.fetchall()

    tconst_list = [tconst[0] for tconst in rows]

    mycursor.close()
    mydb.close()

    loved_films_df = data[data['tconst'].isin(tconst_list)]

    return loved_films_df

# get user liked attributes from db
def get_liked_attributes(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    attribute_fetch = mycursor.fetchall()

    tconst_list = [row[1] for row in attribute_fetch]
    attribute_bin = [row[1:] for row in attribute_fetch]

    mycursor.close()
    mydb.close()

    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    liked_attributes_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst, att_bin in zip(tconst_list, attribute_bin):
        row_values = [tconst]  # Initialize row values with tconst
        for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
            film_att = data[data['tconst'] == tconst]

            if liked == 1:
                row_values.append(film_att[attribute].values[0])
            else:
                row_values.append(None) 
        liked_attributes_df.loc[x] = row_values
        x+=1

    return liked_attributes_df

# get user liked cast from db
def get_liked_cast(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    cast_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    cast = [row[1:] for row in cast_fetch]
    liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

    return liked_cast_df

# get user watchlist from db
def get_watchlist(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))


    watchlist_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    tconst_list = [row[1] for row in watchlist_fetch]
    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    watchlist_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst in tconst_list:
        watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
        x+=1

    return watchlist_df

# calculate % of liked attributes of film in profile
def calculate_percentage(row):
    total_attributes = row['total_likeable']
    total_liked = row['num_liked_atts']
    percentage = total_liked / total_attributes
    return percentage 

# count liked attributes for film
def count_liked(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
    cast = 0
    if not pd.isna(row['cast']):
        cast = len(row['cast'].split(','))
    
    return atts + cast

# CREATE USER PROFILE
def get_user_profile(user_id):
    # get user film preferences from db
    lovedFilms = get_loved_films(user_id)
    likedAtt = get_liked_attributes(user_id)
    likedCast = get_liked_cast(user_id)
    watchlist = get_watchlist(user_id)

    # merge liked cast with liked attribute table
    likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
    likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
    likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

    # format user_profile dataframe
    merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
    user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
    user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
    user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

    # initialise columns to calculate likeage
    user_profile['num_liked_atts'] = 0
    user_profile['likeage'] = 0.0
    # count total liked attributes for each film in profile
    for index, film in user_profile.iterrows():
        tconst = film['tconst']
        user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

    # calculate likeage for each film in profile
    user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
    user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

    return (user_profile, lovedFilms)

# get array of names from inputted dataframe
def extract_names(data):
    names = set()
    for column in data.columns:
        for value in data[column]:
            if value and isinstance(value, str) and value.lower() not in ['none', 'null', 'nan']:
                names.update(value.split(", "))
    return list(names)

# split user profile into likes by group
def collate_liked_groups(user_profile):
    group_dataframes = []

    # Define dictionary to map group names to corresponding columns
    group_columns = {
        'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
        'liked_cast': ['cast', 'likeage'],
        'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
        'liked_genre': ['genres', 'likeage'],
        'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
    }

    # Iterate over the dictionary and create group dataframes
    for group_name, columns in group_columns.items():
        group_df = user_profile[columns].copy()
        # group_df = group_df.dropna(subset=columns[:-1], how='all')
        group_dataframes.append(group_df)

    return group_dataframes

# cosine similarity vector with tf-idf between films in row and column
def create_similarity_vector(row, column):
    tfidf = TfidfVectorizer(stop_words='english')
    row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
    row_soup = row_soup_temp.fillna('')
    row_matrix = tfidf.fit_transform(row_soup)

    column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
    column_soup = column_soup_temp.fillna('')
    column_matrix = tfidf.transform(column_soup)

    return linear_kernel(row_matrix, column_matrix)

# euclidean distance vector for numerical attributes of film
def create_euclidean_vector(row, column):
    scaler = MinMaxScaler()
    row = row.fillna('')
    column = column.fillna(0)

    row_normalized = scaler.fit_transform(row)
    column_normalized = scaler.transform(column)
    distances = euclidean_distances(row_normalized, column_normalized)
    euclidean_matrix = 1 / (1 + distances)
    return euclidean_matrix

# get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
def get_similar_films(vector, exclude):
    mean_similarity = np.mean(vector, axis=1)
    sorted_indices = np.argsort(mean_similarity)[::-1]
    top_N = sorted_indices[:]
    sorted_films = data.iloc[top_N]
    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude['tconst'])]

    return filtered_recommendations

# function to calculate combined recommendations
def get_combined_recommendations(user_profile_groups, similarity_vectors, exclude_films):

    weighted_scores = {}
    
    # scale similarity score in respective vector based on likeage of feature
    for group, attributes in user_profile_groups.items():
        similarity_vector = similarity_vectors[group]
        likeage_array = np.array(list(attributes['likeage'].tolist()))
        weighted_similarity = similarity_vector * likeage_array

        # dictionary of groups and weighted similarity vectors 
        weighted_scores[group] = weighted_similarity

     # combine weighted similarity scores across all groups
    combined_scores = np.sum(list(weighted_scores.values()), axis=0)

    # calculate mean similarity scores
    mean_similarity = np.mean(combined_scores, axis=1)

    # sort the mean similarity scores and retrieve the top N indices
    sorted_indices = np.argsort(mean_similarity)[::-1]

    sorted_films = data.iloc[sorted_indices]

    sorted_films['similarity'] = mean_similarity[sorted_indices] / 5

    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude_films['tconst'])]

    return filtered_recommendations


data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)


In [120]:
# get update user profile and loved films
get_profile = get_user_profile('57')
user_profile = get_profile[0]
lovedFilms = get_profile[1]

# get groups of liked attributes
grouped_likes = collate_liked_groups(user_profile)
liked_plot = grouped_likes[0]
liked_cast = grouped_likes[1]
liked_crew = grouped_likes[2]
liked_genre = grouped_likes[3]
liked_meta = grouped_likes[4]

# Define the list of features excluding 'likeage'
plot_features = [col for col in liked_plot.columns if col != 'likeage']
crew_features = [col for col in liked_crew.columns if col != 'likeage']
cast_features = [col for col in liked_cast.columns if col != 'likeage']
genre_features = [col for col in liked_genre.columns if col != 'likeage']
meta_features = [col for col in liked_meta.columns if col != 'likeage']

# Call the function with the selected columns
plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])

# Similarity vectors for each group
similarity_vectors = {
    'plot': plot_matrix,
    'cast': cast_matrix,
    'crew': crew_matrix,
    'genre': genre_matrix,
    'meta': meta_matrix
}

# meta_matrix
user_profile_groups = {
    'plot': liked_plot,
    'cast': liked_cast,
    'crew': liked_crew,
    'genre': liked_genre,
    'meta': liked_meta
}

user_profile
# recommended = get_combined_recommendations(user_profile_groups, similarity_vectors, lovedFilms)
# get_similar_films(genre_matrix, lovedFilms)[:20]


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,likeage
0,tt0367027,Shortbus,"In post-9/11 New York City, an eclectic group ...",6.4,"Comedy,Drama,Romance",101,2006,"Peter Stickles, PJ DeBoy, Paul Dawson, Sook-Yi...",John Cameron Mitchell,Frank G. DeMarco,None,"Howard Gertler, Tim Perell",Brian A. Kates,Yo La Tengo,1.000000
1,tt0120689,The Green Mile,A supernatural tale set on death row in a Sout...,8.6,"Crime,Drama,Fantasy",189,1999,"Tom Hanks, Michael Clarke Duncan, David Morse,...",Frank Darabont,David Tattersall,Stephen King,David Valdes,Richard Francis-Bruce,Thomas Newman,1.000000
2,tt0483835,Four Windows,"They want to get away from each other, but the...",4.6,Drama,80,2006,"Thorsten Merten, Frank Droese, Margarita Broic...",Christian Moris Müller,Jürgen Jürges,None,Philipp Budweg,Maja Stieghorst,Chandra Fleig,1.000000
3,tt0057043,The Happy Musketeers,The Big Bad Wolf and the Stinky Little Skunk f...,5.1,"Adventure,Comedy,Drama",86,1961,"Elmo Michel, Santanón, Ariadne Welter, Marina ...",Roberto Rodríguez,Alex Phillips,Manuel R. Ojeda,Aurelio García Yévenes,José W. Bustos,Sergio Guerrero,1.000000
4,tt0373921,Mother of the Bride,This magnificently madcap comedy classic stars...,7.5,"Comedy,Drama,Family",102,1963,"Imad Hamdi, Hasan Youssef, Samira Ahmed, Madih...",Atef Salem,Masud Isa,"Abdel Hai Adib, Abdel Hamid Gouda",Naguib Khoury,Hussein Ahmed,None,1.000000
5,tt0178576,L'hiver bleu,L’Hiver bleu is an Abitibian chronicle that pr...,5.7,Drama,82,1979,"Michel Chenier, Réjean Roy, Christiane Lévesqu...",André Blanchard,Alain Dupras,Jeanne-Mance Delisle,Marguerite Duparc,None,"Abbittibbi, Les Anonymes d'Évias",1.000000
6,tt1512235,Super,After his wife falls under the influence of a ...,6.7,"Action,Comedy,Crime",96,2010,"Rainn Wilson, Elliot Page, Kevin Bacon, Liv Tyler",James Gunn,Steve Gainer,None,"Miranda Bailey, Ted Hope",Cara Silverman,Tyler Bates,1.000000
7,tt7454396,A Mother,"After getting released from prison, Eeva retur...",5.8,Drama,91,2019,"Matti Pajulahti, Jaana Saarinen, Helmi-Leena N...",Samppa Batal,None,Krista Hannula,"Maarit Laaksoharju, Miikka Pakarinen",Tuomas Kohtamäki,Antti L.S. Ikonen,1.000000
8,tt0049103,None,None,NaN,"Horror,Sci-Fi",None,None,NaN,None,None,None,None,None,None,0.066667
9,tt0083067,None,None,NaN,"Comedy,Musical",None,None,NaN,None,None,None,None,None,None,0.066667


In [125]:
# get user watchlist from db
def get_watchlist(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))


    watchlist_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    tconst_list = [row[1] for row in watchlist_fetch]
    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'poster']
    watchlist_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst in tconst_list:
        watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
        x+=1

    return watchlist_df

get_watchlist('57')

,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster
0,tt0049434,The Last Wagon,When a handful of settlers survive an Apache a...,7.0,"Drama,Western",99,1956,"Richard Widmark, Tommy Rettig, Felicia Farr, S...",Delmer Daves,Wilfrid M. Cline,"James Edward Grant, Gwen Bagni",William B. Hawks,None,Lionel Newman,/2FHbwL1RZCeAsBRSBVuxYRj9Mhl.jpg
1,tt0051378,Elevator to the Gallows,A self-assured businessman murders his employe...,7.9,"Crime,Drama,Thriller",91,1958,"Maurice Ronet, Georges Poujouly, Jeanne Moreau...",Louis Malle,Henri Decaë,"Roger Nimier, Noël Calef",Jean Thuillier,None,Miles Davis,/dLbf8LtbLQFM7KUaVgwkfKIPqXu.jpg
2,tt0052944,Jet Storm,Crisis in the air: A passenger aboard a commer...,6.5,Thriller,99,1959,"Richard Attenborough, Stanley Baker, Bernard B...",Cy Endfield,Jack Hildyard,Sigmund Miller,Steven Pallos,Oswald Hafenrichter,Thomas Rajna,/9mdpGhwje48y0RM5Bq1AGKe91jk.jpg
3,tt0054286,Cruel Story of Youth,A budding gangster enthralls a freeloading you...,7.0,"Crime,Drama",96,1960,"Yûsuke Kawazu, Fumio Watanabe, Miyuki Kuwano, ...",Nagisa Ôshima,Takashi Kawamata,None,Tomio Ikeda,Keiichi Uraoka,Riichirô Manabe,/hLPbvJEGGnOEWCUl7IhNSETDXs9.jpg
4,tt0058994,Bruce's Fingers,After malicious gangsters capture Bruce Wong's...,4.9,"Action,Crime,Drama",91,1976,"Bruce Le, Michael Wai-Man Chan, Lieh Lo, Nora ...",Joseph Velasco,Hak-Wai Ho,Yun-Ping Wu,Liu En-Ze,Nan Hsiao,Lawrence Chan,/iJF8yMqI14LndLwGpfWiOuoCsvu.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,tt6241768,Snowbound,A group of erotic party attendees wake up nake...,3.2,"Horror,Mystery",93,2017,"Drew Hale, Theodore Bouloukos, Anya Bay, Laura...",Olia Oparina,Halyna Hutchins,None,Kristina Keyta,"Slava Denisov, Iaroslav Denisov",George Sarah,/ihdVBKEgSLOjoAYQcFwYeMDcIma.jpg
66,tt7152766,Sapthagiri Express,"Sapthagiri is a happy go lucky aspiring actor,...",7.4,Comedy,131,2016,"Sapthagiri, Mohammad Ali, Shakalaka Shankar, R...",Arun Pawar,C. Ram Prasad,None,Ravi Kirane,"Goutham Raju, Gautham Raju",Vijay Bulganin,/yKyKLzLPVsZVrmrd5bh6Ou6w2QK.jpg
67,tt7219324,The House by the Sea,By a little bay near Marseille lies a pictures...,6.5,"Comedy,Drama",107,2017,"Jean-Pierre Darroussin, Gérard Meylan, Jacques...",Robert Guédiguian,Pierre Milon,Serge Valletti,Marc Bordure,Bernard Sasia,None,/9EpuaE0QmNkOxhHzfUfsuzXwo60.jpg
68,tt7543152,Best of Luck Laalu,"Story of a boy named Laalu, a student of tenth...",7.4,Comedy,131,2017,"Kurush Deboo, Smit Ganatra, Muni Jha, Supriya ...",Vipul Mehta,Pushkar Singh,Amatya Goradia,Rashmin Majithia,None,"Salil Amrute, Jigar Saraiya",/rECPjLdLvokP4MNjKrrVMRy7bTQ.jpg


In [122]:
from sklearn.feature_extraction.text import CountVectorizer


# cosine similarity vector with tf-idf between films in row and column
def create_soup(x, features):
    soup_parts = [1 if x[feature] else 0 for feature in features]  # Encode genre presence as binary values
    return soup_parts

# join and concatenate inputted columns
def create_soup(x, features):
    soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
    return ' '.join(soup_parts)


def create_similarity_vector(row, column):
    tfidf = CountVectorizer(stop_words='english')
    tfidf = TfidfVectorizer(stop_words='english')

    row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
    row_soup = row_soup_temp.fillna('')
    row_matrix = tfidf.fit_transform(row_soup)

    column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
    column_soup = column_soup_temp.fillna('')
    column_matrix = tfidf.transform(column_soup)

    return linear_kernel(row_matrix, column_matrix)


def get_similar_films(vector):
    mean_similarity = np.mean(vector, axis=1)
    sorted_indices = np.argsort(mean_similarity)[::-1]
    top_N = sorted_indices[:]
    sorted_films = data.iloc[top_N]
    # filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude['tconst'])]

    return sorted_films

# genre_features = [col for col in liked_genre.columns if col != 'likeage']
# liked_genre = liked_genre.drop(columns="likeage")
x = liked_genre.apply(lambda x: create_soup(x, liked_genre.columns), axis=1)

x

genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
get_similar_films(genre_matrix)[:25]


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
5936,tt0104119,Dolly Dearest,An American family moves to Mexico to fabricat...,4.7,"Comedy,Drama,Horror",93,1991,"Sam Bottoms, Rip Torn, Chris Demetral, Denise ...",Maria Lease,Eric D. Andersen,"Rod Nave, Peter Sutcliffe",Daniel Cady,None,Mark Snow,/fp5o7DIp31R3T7SNzPGVKgyyTpu.jpg,15
17260,tt0096343,Unmasked Part 25,Jackson is a lonely serial killer who is reall...,5.8,"Comedy,Drama,Horror",85,1988,"Gregory Cox, Edward Brayshaw, Fiona Evans, Deb...",Anders Palm,John de Borman,"Mark Cutforth, William Shakespeare",None,Brian Peachey,Julian Wastall,/acGNEpFLYiSTArlG1utBhpqWs9f.jpg,15
13175,tt1653881,Help of God,"""Kamisama Help!"" takes place at an abandoned s...",6.0,"Comedy,Drama,Horror",87,2010,"Kazuki Katô, Kento Kaku, Takahiro Hôjô, Aimi S...",Shota Sasaki,Chigi Kanbe,Ichiro Anabuki,"Kenji Nakanishi, Hitoshi Nunokawa",None,Kuniyuki Morohashi,/4xQMwKsFq4QSjUHi8eD6Dz2lPPS.jpg,15
13169,tt14773734,Bomma Adirindi Dhimma Tirigindi,"After five years, a group of college girls pla...",6.8,"Comedy,Drama,Horror",109,2021,"Arjun Kalyan, Shakalaka Shankar, Avantika, San...",Kumar Kota,Phanindra Varma Alluri,V.S. Rao,L. Madhu,None,Bhagawath,/lA1WD2HvIsV7qA2KlSMC1Sp0EIo.jpg,15
9444,tt10441822,The Nowhere Inn,When St. Vincent sets out to make a documentar...,5.7,"Comedy,Drama,Horror",91,2020,"Ezra Buzzington, Toko Yasuda, Chris Aquilino, ...",Bill Benz,Minka Farthing-Kohl,Carrie Brownstein,"Lana Kim, Jett Steiger",Ali Greer,None,/saXbsVa6eylbaOCXpwGZVjn1fOd.jpg,15
10528,tt0432006,Puppy,"Attempting suicide, sultry but down-on-her-luc...",6.0,"Comedy,Drama,Horror",95,2005,"Bernard Curry, Terence Donovan, Nadia Townsend...",Kieran Galvin,Tom Gleeson,None,Melissa Beauford,Mark Atkin,Karel Segers,/oTm4rx4MfOIVj9Po1GAKohJXT4X.jpg,15
17414,tt0087289,Frankenstein 90,French cybernetics genius Victor Frankenstein ...,4.5,"Comedy,Drama,Horror",92,1984,"Jean Rochefort, Eddy Mitchell, Fiona Gélin, He...",Alain Jessua,William Lubtchansky,"Paul Gégauff, Mary Shelley",None,Hélène Plemiannikov,Armando Trovajoli,/cNZxL0yeDZ37YVCf05us8nbhhjF.jpg,15
510,tt0104139,Dr. Giggles,"In 1957, Evan Rendell flees after his father i...",5.3,"Comedy,Drama,Horror",95,1992,"Larry Drake, Cliff De Young, Glenn Quinn, Holl...",Manny Coto,Robert Draper,Graeme Whifler,Stuart M. Besser,Debra Neil-Fisher,Brian May,/15TOuWoaJ5C0lLcmRTQz5Lasnd.jpg,16
18101,tt5810368,Deadly Detention,Five archetypal teens serving detention find t...,3.4,"Comedy,Drama,Horror",90,2017,"Henrique Zaga, Coy Stewart, Alex Frnka, Sarah ...",Blair Hayes,Kristoffer Carrillo,"Casie Tabanou, Ali Spuck",Valentina Bove,None,Matthew Puckett,/ket8bzYI0ryOV1JhjsmWIfvs7xf.jpg,15
3787,tt14476124,Ghosty,An inspector embarks on a mission to track dow...,4.3,"Comedy,Drama,Horror",139,2023,"Madhan Babu, Yogi Babu, Kpy Bala, Kajal Aggarwal",Kalyaan,Jacob Rathinaraj,None,"Jayaram G., Sudhan Sundaram",Vijay Velukutty,Sam C.S.,/8VPfGQDBYA3fldv2C5Cxh3gQ3nR.jpg,15


In [92]:
liked_genres_df = liked_genre.drop(columns="likeage")
liked_genres_df
# get_similar_films(genre_matrix)

,genres
0,"Biography,Drama,Romance"
1,Comedy
2,"Comedy,Crime"
3,Drama
4,"Comedy,Horror,Thriller"
...,...
67,"Action,Drama,Sport"
68,Adventure
69,"Action,Comedy,Horror"
70,"Family,Fantasy"
